In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_moons

from tensorflow.keras.optimizers import SGD

from keras.models import Sequential, load_model
from keras.layers import Dense

from keras.callbacks import TensorBoard
from keras.callbacks import ModelCheckpoint

from sklearn.model_selection import train_test_split
from os.path import exists
import datetime

In [ ]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [ ]:
# Load moons data
X,y = make_moons(n_samples=2000, noise=0.1)
print(X)

In [ ]:
# Plot the moons data points
df = pd.DataFrame(dict(x=X[:,0],y=X[:,1],label=y))
colors = {0:'red', 1: 'blue'}
fig, ax = plt.subplots()
grouped = df.groupby('label')
for key, group in grouped:
    group.plot(ax=ax, kind='scatter',x='x', y='y', label=key, color=colors[key])

In [ ]:
print(df)

In [ ]:
#X and y for the model

x_model = df[['x','y']]
y_model = df.label

In [ ]:
print(x_model)

In [ ]:
print(y_model)

In [ ]:
# Create MLP
def create_model(lr_rate):
    model = Sequential()
    model.add(Dense(64, activation = 'relu'))
    model.add(Dense(1, activation = 'sigmoid'))
    opt = SGD(learning_rate=lr_rate)
    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
# Logistic regression function 
def logistic_regression(name, lr_rate, batch):
    
    #Tensorboard
    log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    tensorboard = TensorBoard(log_dir=log_dir, histogram_freq=1, write_graph=True, write_images=True, update_freq='epoch')
    
    #Checkpoint
    chekpoint_path= name + ".Q4.weights.best.hdf5"
    checkpoint = ModelCheckpoint(chekpoint_path, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
    
    callbacks = [tensorboard, checkpoint]
    
    if exists(chekpoint_path):
        # Load model
        model = load_model(chekpoint_path)
        # Finding the epoch index from which we are resuming
        initial_epoch = 0
    else:
        model = create_model(lr_rate)
        initial_epoch = 0
    
    # Fit the model
    x_train, x_test, y_train, y_test = train_test_split(x_model, y_model, test_size=0.33, random_state=1)
    model.fit(x_train, y_train, validation_data=(x_test, y_test), batch_size=batch, epochs=100, callbacks=callbacks, initial_epoch=initial_epoch)
    
    # Print training accuracy & test accuracy
    _, train_acc = model.evaluate(x_train, y_train, verbose=0)
    _, test_acc = model.evaluate(x_test, y_test, verbose=0)
    print('Train: %.3f, Test: %.3f' % (train_acc, test_acc))

In [ ]:
logistic_regression("first", 0.01, 16)

In [ ]:
# Tweaking hyperparameters
logistic_regression("second", 0.01, 32)

In [ ]:
# Tweaking hyperparameters
logistic_regression("third", 0.2, 16)